<a href="https://colab.research.google.com/github/dakilaledesma/herbarium_model_testing/blob/main/H2022_GPU_DualNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/H2022/RCNN_PP/pls224p56_Orchidaceae_train.zip -d /content/
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 168 ms, sys: 41.9 ms, total: 209 ms
Wall time: 25.1 s


In [3]:
from glob import glob
import os
cat_to_int = {}
for i, f in enumerate(sorted(glob("pls224p56_Orchidaceae_train/*"))):
  cat = os.path.basename(f)
  cat_to_int[cat] = i

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate
import numpy as np

classes = 300

orig_model = ResNet50V2(weights='imagenet', include_top=False)
lg_model = ResNet50V2(weights='imagenet', include_top=False)


for ln in [layer.name for layer in lg_model.layers]:
    lg_model.get_layer(name=ln)._name = f"lg_{ln}"

out = Concatenate()([orig_model.output, lg_model.output])
out = Dense(2048, activation='relu')(out)
out = Dense(classes, activation='softmax', name='predictions')(out)

model = Model(inputs=[orig_model.input, lg_model.input], outputs=out)

model.compile(optimizer='adam', loss='categorical_crossentropy')